In [1]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Data Markings

### Creating Objects With Data Markings

To create an object with a (predefined) TLP marking to an object, just provide it as a keyword argument to the constructor. The TLP markings can easily be imported from python-stix2.

In [2]:
from stix2 import Indicator, TLP_AMBER

indicator = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs=TLP_AMBER)
print(indicator)

{
    "type": "indicator",
    "id": "indicator--e459c8d7-09aa-41c5-968d-66bf8610687e",
    "created": "2017-09-15T16:32:04.495Z",
    "modified": "2017-09-15T16:32:04.495Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:04.495379Z",
    "object_marking_refs": [
        "marking-definition--f88d31f6-486f-44da-b317-01333bde0b82"
    ]
}


If you’re creating your own marking (for example, a ``Statement`` marking), first create the statement marking:

In [3]:
from stix2 import MarkingDefinition, StatementMarking

marking_definition = MarkingDefinition(                                    
    definition_type="statement",                                            
    definition=StatementMarking(statement="Copyright 2017, Example Corp")
)
print(marking_definition)

{
    "type": "marking-definition",
    "id": "marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0",
    "created": "2017-09-15T16:32:07.947008Z",
    "definition_type": "statement",
    "definition": {
        "statement": "Copyright 2017, Example Corp"
    }
}


Then you can add it to an object as it’s being created (passing either full object or the the ID as a keyword argument, like with relationships).

In [4]:
indicator2 = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs=marking_definition)
print(indicator2)

{
    "type": "indicator",
    "id": "indicator--06eea9e3-ea6d-418b-a517-4b1fff96f4aa",
    "created": "2017-09-15T16:32:10.859Z",
    "modified": "2017-09-15T16:32:10.859Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:10.859197Z",
    "object_marking_refs": [
        "marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0"
    ]
}


In [5]:
indicator3 = Indicator(labels=["malicious-activity"],
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      object_marking_refs="marking-definition--f88d31f6-486f-44da-b317-01333bde0b82")
print(indicator3)

{
    "type": "indicator",
    "id": "indicator--94457e6e-4282-4355-8f2a-a59be2192b31",
    "created": "2017-09-15T16:32:13.322Z",
    "modified": "2017-09-15T16:32:13.322Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:13.322982Z",
    "object_marking_refs": [
        "marking-definition--f88d31f6-486f-44da-b317-01333bde0b82"
    ]
}


Granular markings work in the same way, except you also need to provide a full granular-marking object (including the selector).

In [6]:
from stix2 import Malware, TLP_WHITE

malware = Malware(name="Poison Ivy",
                  labels=['remote-access-trojan'],
                  description="A ransomware related to ...",
                  granular_markings=[
                      {
                          "selectors": ["description"],
                          "marking_ref": marking_definition
                      },
                      {
                          "selectors": ["name"],
                          "marking_ref": TLP_WHITE
                      }
                  ])
print(malware)

{
    "type": "malware",
    "id": "malware--c91e0491-ea7f-46e4-85cd-90004da16b8a",
    "created": "2017-09-15T16:32:15.801Z",
    "modified": "2017-09-15T16:32:15.801Z",
    "name": "Poison Ivy",
    "description": "A ransomware related to ...",
    "labels": [
        "remote-access-trojan"
    ],
    "granular_markings": [
        {
            "marking_ref": "marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0",
            "selectors": [
                "description"
            ]
        },
        {
            "marking_ref": "marking-definition--613f2e26-407d-48c7-9eca-b8e91df99dc9",
            "selectors": [
                "name"
            ]
        }
    ]
}


Make sure that the selector is a field that exists and is populated on the object, otherwise this will cause an error:

In [7]:
Malware(name="Poison Ivy",
        labels=['remote-access-trojan'],
        description="A ransomware related to ...",
        granular_markings=[
            {
                "selectors": ["title"],
                "marking_ref": marking_definition
            }
        ])

InvalidSelectorError: Selector title in Malware is not valid!

### Adding Data Markings To Existing Objects

Both object markings and granular markings can also be added to STIX objects which have already been created. Doing so will create a new version of the object (note the updated ``modified`` time).

In [16]:
from stix2.markings import add_markings

indicator4 = add_markings(indicator, marking_definition.id)
print(indicator4)

{
    "type": "indicator",
    "id": "indicator--e459c8d7-09aa-41c5-968d-66bf8610687e",
    "created": "2017-09-15T16:32:04.495Z",
    "modified": "2017-09-15T16:51:40.883Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:04.495379Z",
    "object_marking_refs": [
        "marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0",
        "marking-definition--f88d31f6-486f-44da-b317-01333bde0b82"
    ]
}


You can also remove specific markings from STIX objects. This will also create a new version of the object.

In [21]:
from stix2.markings import remove_markings

indicator5 = remove_markings(indicator4, marking_definition.id)
print(indicator5)

{
    "type": "indicator",
    "id": "indicator--e459c8d7-09aa-41c5-968d-66bf8610687e",
    "created": "2017-09-15T16:32:04.495Z",
    "modified": "2017-09-15T16:59:35.160Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:04.495379Z",
    "object_marking_refs": [
        "marking-definition--f88d31f6-486f-44da-b317-01333bde0b82"
    ]
}


The markings on an object can be replaced with a different set of markings:

In [22]:
from stix2 import TLP_GREEN
from stix2.markings import set_markings

indicator6 = set_markings(indicator5, [TLP_GREEN.id, marking_definition.id])
print(indicator6)

{
    "type": "indicator",
    "id": "indicator--e459c8d7-09aa-41c5-968d-66bf8610687e",
    "created": "2017-09-15T16:32:04.495Z",
    "modified": "2017-09-15T16:59:42.906Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:04.495379Z",
    "object_marking_refs": [
        "marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0",
        "marking-definition--34098fce-860f-48ae-8e50-ebd3cc5e41da"
    ]
}


STIX objects can also be cleared of all markings:

In [23]:
from stix2.markings import clear_markings

indicator7 = clear_markings(indicator5)
print(indicator7)

{
    "type": "indicator",
    "id": "indicator--e459c8d7-09aa-41c5-968d-66bf8610687e",
    "created": "2017-09-15T16:32:04.495Z",
    "modified": "2017-09-15T17:04:48.386Z",
    "labels": [
        "malicious-activity"
    ],
    "pattern": "[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    "valid_from": "2017-09-15T16:32:04.495379Z"
}


All of these functions can be used for granular markings by passing in a list of selectors. Note that they will create new versions of the objects.

### Evaluating Data Markings

You can get a list of the object markings on a STIX object:

In [24]:
from stix2.markings import get_markings

get_markings(indicator6)

['marking-definition--674e3f0f-8547-41df-80b4-784eb4c5b4d0',
 'marking-definition--34098fce-860f-48ae-8e50-ebd3cc5e41da']

You can also get a list of granular markings by passing a list of selectors to ``get_markings``:

In [25]:
get_markings(malware, 'name')

['marking-definition--613f2e26-407d-48c7-9eca-b8e91df99dc9']

You can also check if an object is marked by a specific markings. Again, for granular markings, pass in the selector or list of selectors.

In [11]:
from stix2.markings import is_marked

is_marked(indicator, TLP_AMBER.id)

True

In [13]:
is_marked(malware, TLP_WHITE.id, 'name')

True

In [14]:
is_marked(malware, TLP_WHITE.id, 'description')

False